## Types of explode functions
1. explode()
2. explore_outer()
3. posexplode()
4. posexplode_outer()

In [15]:
# Create SparkSession and Prepare sample Data
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark-by-examples').getOrCreate()

arrayData = [
        ('James',['Java','Scala'],{'hair':'black','eye':'brown'}),
        ('Michael',['Spark','Java',None],{'hair':'brown','eye':None}),
        ('Robert',['CSharp',''],{'hair':'red','eye':''}),
        ('Washington',None,None),
        ('Jefferson',['1','2'],{})]

df = spark.createDataFrame(data=arrayData, schema = ['name','knownLanguages','properties'])
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- knownLanguages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+----------+-------------------+-----------------------------+
|name      |knownLanguages     |properties                   |
+----------+-------------------+-----------------------------+
|James     |[Java, Scala]      |{eye -> brown, hair -> black}|
|Michael   |[Spark, Java, NULL]|{eye -> NULL, hair -> brown} |
|Robert    |[CSharp, ]         |{eye -> , hair -> red}       |
|Washington|NULL               |NULL                         |
|Jefferson |[1, 2]             |{}                           |
+----------+-------------------+-----------------------------+



In [4]:
display(df.collect())

[Row(name='James', knownLanguages=['Java', 'Scala'], properties={'eye': 'brown', 'hair': 'black'}),
 Row(name='Michael', knownLanguages=['Spark', 'Java', None], properties={'eye': None, 'hair': 'brown'}),
 Row(name='Robert', knownLanguages=['CSharp', ''], properties={'eye': '', 'hair': 'red'}),
 Row(name='Washington', knownLanguages=None, properties=None),
 Row(name='Jefferson', knownLanguages=['1', '2'], properties={})]

In [6]:
# Apply the explode on knownLanguages column
#explode function can convert each element in an array of column into each element as a row
from pyspark.sql.functions import explode, col
df1 = df.withColumn('knownLanguages',explode(col('knownLanguages')))
print(df1.printSchema())
print(df1.show(truncate=False))

root
 |-- name: string (nullable = true)
 |-- knownLanguages: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

None
+---------+--------------+-----------------------------+
|name     |knownLanguages|properties                   |
+---------+--------------+-----------------------------+
|James    |Java          |{eye -> brown, hair -> black}|
|James    |Scala         |{eye -> brown, hair -> black}|
|Michael  |Spark         |{eye -> NULL, hair -> brown} |
|Michael  |Java          |{eye -> NULL, hair -> brown} |
|Michael  |NULL          |{eye -> NULL, hair -> brown} |
|Robert   |CSharp        |{eye -> , hair -> red}       |
|Robert   |              |{eye -> , hair -> red}       |
|Jefferson|1             |{}                           |
|Jefferson|2             |{}                           |
+---------+--------------+-----------------------------+

None


In [11]:
## selecting the name and knownLanguages as explode columns
# explode on array column example
df.select(df.name,explode(col('knownLanguages'))).show()


+---------+------+
|     name|   col|
+---------+------+
|    James|  Java|
|    James| Scala|
|  Michael| Spark|
|  Michael|  Java|
|  Michael|  NULL|
|   Robert|CSharp|
|   Robert|      |
|Jefferson|     1|
|Jefferson|     2|
+---------+------+



In [12]:
# explode on map column example
# on the map column it will create two separate rows for key and value
df.select(df.name,explode(col('properties'))).show()


+-------+----+-----+
|   name| key|value|
+-------+----+-----+
|  James| eye|brown|
|  James|hair|black|
|Michael| eye| NULL|
|Michael|hair|brown|
| Robert| eye|     |
| Robert|hair|  red|
+-------+----+-----+



In [13]:
## 2. explode_outer() – Create rows for each element in an array or map.
# PySpark SQL explode_outer(e: Column) function is used to create a row for each element in the array or map column.
# Unlike explode, if the array or map is null or empty, explode_outer returns null.
from pyspark.sql.functions import explode_outer
df.select(df.name,explode_outer(col('knownLanguages'))).show()

+----------+------+
|      name|   col|
+----------+------+
|     James|  Java|
|     James| Scala|
|   Michael| Spark|
|   Michael|  Java|
|   Michael|  NULL|
|    Robert|CSharp|
|    Robert|      |
|Washington|  NULL|
| Jefferson|     1|
| Jefferson|     2|
+----------+------+



In [14]:
df.select(df.name,explode_outer(col('properties'))).show()

+----------+----+-----+
|      name| key|value|
+----------+----+-----+
|     James| eye|brown|
|     James|hair|black|
|   Michael| eye| NULL|
|   Michael|hair|brown|
|    Robert| eye|     |
|    Robert|hair|  red|
|Washington|NULL| NULL|
| Jefferson|NULL| NULL|
+----------+----+-----+



In [17]:
## 3. posexplode() – explode array or map elements to rows
# posexplode(e: Column) creates a row for each element in the array.
# creates two columns “pos’ to hold the position of the array element and the ‘col’ to hold the actual array value. 
# For map it will create 3 columns 
# This will ignore elements that are null or empty. 

from pyspark.sql.functions import posexplode
df.select(df.name,posexplode(col('properties'))).show()

+-------+---+----+-----+
|   name|pos| key|value|
+-------+---+----+-----+
|  James|  0| eye|brown|
|  James|  1|hair|black|
|Michael|  0| eye| NULL|
|Michael|  1|hair|brown|
| Robert|  0| eye|     |
| Robert|  1|hair|  red|
+-------+---+----+-----+



In [18]:
df.select(df.name,posexplode(col('knownLanguages'))).show()

+---------+---+------+
|     name|pos|   col|
+---------+---+------+
|    James|  0|  Java|
|    James|  1| Scala|
|  Michael|  0| Spark|
|  Michael|  1|  Java|
|  Michael|  2|  NULL|
|   Robert|  0|CSharp|
|   Robert|  1|      |
|Jefferson|  0|     1|
|Jefferson|  1|     2|
+---------+---+------+



In [19]:
## 4. posexplode_outer() – explode array or map columns to rows.
# It is the combination of explode_outer and position
# # This will show elements that are null or empty. 
from pyspark.sql.functions import posexplode_outer
df.select(df.name,posexplode_outer(col('knownLanguages'))).show()

+----------+----+------+
|      name| pos|   col|
+----------+----+------+
|     James|   0|  Java|
|     James|   1| Scala|
|   Michael|   0| Spark|
|   Michael|   1|  Java|
|   Michael|   2|  NULL|
|    Robert|   0|CSharp|
|    Robert|   1|      |
|Washington|NULL|  NULL|
| Jefferson|   0|     1|
| Jefferson|   1|     2|
+----------+----+------+



In [20]:
df.select(df.name,posexplode_outer(col('properties'))).show()

+----------+----+----+-----+
|      name| pos| key|value|
+----------+----+----+-----+
|     James|   0| eye|brown|
|     James|   1|hair|black|
|   Michael|   0| eye| NULL|
|   Michael|   1|hair|brown|
|    Robert|   0| eye|     |
|    Robert|   1|hair|  red|
|Washington|NULL|NULL| NULL|
| Jefferson|NULL|NULL| NULL|
+----------+----+----+-----+



In [25]:
df2 = df.withColumn('knownLanguages',explode(col('knownLanguages')))
print(df2.printSchema())
print(df2.show(truncate=False))

root
 |-- name: string (nullable = true)
 |-- knownLanguages: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

None
+---------+--------------+-----------------------------+
|name     |knownLanguages|properties                   |
+---------+--------------+-----------------------------+
|James    |Java          |{eye -> brown, hair -> black}|
|James    |Scala         |{eye -> brown, hair -> black}|
|Michael  |Spark         |{eye -> NULL, hair -> brown} |
|Michael  |Java          |{eye -> NULL, hair -> brown} |
|Michael  |NULL          |{eye -> NULL, hair -> brown} |
|Robert   |CSharp        |{eye -> , hair -> red}       |
|Robert   |              |{eye -> , hair -> red}       |
|Jefferson|1             |{}                           |
|Jefferson|2             |{}                           |
+---------+--------------+-----------------------------+

None


In [26]:
df3 = df.withColumn('properties',posexplode_outer(col('properties')))
print(df3.printSchema())
print(df3.show(truncate=False))

AnalysisException: [UDTF_ALIAS_NUMBER_MISMATCH] The number of aliases supplied in the AS clause does not match the number of columns output by the UDTF. Expected 3 aliases, but got properties. Please ensure that the number of aliases provided matches the number of columns output by the UDTF.